In [ ]:
%cd /content/drive/MyDrive/attacks/
!ls

In [ ]:
# load function taken from https://github.com/itaygal/RS_TrueReputation/
import re
import copy
import os

"""load rating .csv file and save results to given data structures 
   Args:
       dataset_path: path to movielens 100k rating file
       user_movie_ratings: dic of user to a dic of movie to a rating. user_movie_ratings[user_id][movie_id] = rating
       movie_user_ratings:  dic of movie to a dic of user to a rating. movie_user_ratings[movie_id][user_id] = rating
       movies: set of all movie names
   Returns:
       None.
"""
def load(dataset_path, user_movie_ratings, movie_user_ratings, movies):
    # user id | item id | rating | timestamp
    rating_match = re.compile("\D*(\d+)\D*(\d+)\D*(\d+)\D*(\d+)")
    with open(dataset_path, 'r') as dataset_file:
        for rating_line in dataset_file:
            m = rating_match.match(rating_line)
            if m:
                user_id = m.group(1)
                movie_id = m.group(2)
                rating = m.group(3)
                timestamp = m.group(4)
                if user_id not in user_movie_ratings:
                    user_movie_ratings[user_id] = {}
                user_movie_ratings[user_id][movie_id] = (int(rating), int(timestamp))
                if movie_id not in movie_user_ratings:
                    movie_user_ratings[movie_id] = {}
                movies.add(movie_id)
                movie_user_ratings[movie_id][user_id] = (int(rating), int(timestamp))

In [ ]:

user_movie_ratings = {}  # dic of user to a dic of movie to a rating. user_movie_ratings[user_id][movie_id] = rating
movie_user_ratings = {}  # dic of movie to a dic of user to a rating. movie_user_ratings[movie_id][user_id] = rating
movies = set() # set of all movie names
# load rating .csv file into data structures
load("/content/drive/MyDrive/attacks/ml-100k/u.data", user_movie_ratings, movie_user_ratings, movies)
# load item information .csv release year into movie release year


In [ ]:
from tqdm import tqdm
Movies={}
for m in tqdm(movie_user_ratings):
  Movies[m]=[]
  Movies[m].append(len(movie_user_ratings[m]))
  avg_rating=0
  for i in movie_user_ratings[m]:
    avg_rating+=movie_user_ratings[m][i][0]
  Movies[m].append(avg_rating/len(movie_user_ratings[m]))



100%|██████████| 1682/1682 [00:00<00:00, 17737.98it/s]


In [ ]:
from collections import defaultdict
import numpy as np
import random
userProfile=defaultdict(dict)
itemProfile = defaultdict(dict)
timeProfile= defaultdict(dict)
random_time=[]
for user in user_movie_ratings:
  for item in user_movie_ratings[user]:
    userProfile[int(user)][int(item)]=int(user_movie_ratings[user][item][0])
    itemProfile[int(item)][int(user)]=int(user_movie_ratings[user][item][0])
    timeProfile[int(user)][int(item)]=int(user_movie_ratings[user][item][1])
    random_time.append(int(user_movie_ratings[user][item][1]))

In [ ]:
# attack functions are modified from https://github.com/Coder-Yu/SDLib
############################################### config ###################################

outputDir = "/content/drive/MyDrive/attack_datasets/Movielens1M/bandwagon/"
attackSize = 0.05
fillerSize = 0.05
selectedSize = 0.005
targetCount = 100
targetScore = 4.0
threshold = 3.0
maxScore = 4.0
minScore = 1.0
minCount = 50
maxCount = 200
linkSize = 0.001


itemList = []

spamProfile = defaultdict(dict)
spamItem = defaultdict(list)  # items rated by spammers
spamTimeProfile = defaultdict(dict)
targetItems = []
itemAverage = {}
startUserID = 0


def getAverageRating():
    for itemID in itemProfile:
        li = itemProfile[itemID].values()
        itemAverage[itemID] = float(sum(li)) / len(li)


def selectTarget():
    print('Selecting target items...')
    
    for i in itemProfile.keys():
      itemList.append(i)
    itemList.sort()
    while len(targetItems) < targetCount:
        # generate a target order at random
        target = np.random.randint(len(itemList))

        if len(itemProfile[itemList[target]]) < maxCount and len(itemProfile[itemList[target]]) > minCount \
                and itemList[target] not in targetItems \
                and itemAverage[itemList[target]] <= threshold:
            targetItems.append(itemList[target])
            #print(itemList[target], ' ', itemAverage[itemList[target]])

############################################### config ###################################

In [ ]:
def generateLabels(filename):
    labels = []
    path = outputDir + filename
    with open(path, 'w') as f:
        for user in spamProfile:
            labels.append(str(user)+' 1\n')
        for user in userProfile:
            labels.append(str(user)+' 0\n')
        f.writelines(labels)
    print('User profiles have been output')


def generateProfiles(filename):
    ratings = []
    path = outputDir+filename
    with open(path, 'w') as f:
        for user in userProfile:
            for item in userProfile[user]:
                ratings.append(str(user)+' '+str(item)+' ' +
                               str(userProfile[user][item])+' '+str(timeProfile[user][item])+'\n')

        for user in spamProfile:
            for item in spamProfile[user]:
                ratings.append(str(user) + ' ' + str(item) + ' ' +
                               str(spamProfile[user][item])+' '+str(spamTimeProfile[user][item])+'\n')
        print(len(spamProfile))
        f.writelines(ratings)
    print('User labels have been output')

In [ ]:

############################################## average attack ##########################################
def average_attack(startID=0):
    print('Modeling average attack...')
    startUserID = len(userProfile) if startID == 0 else startID
    for _ in range(int(len(userProfile)*attackSize)):
        fillerItems = getFillerItems()
        for item in fillerItems:
            spamProfile[startUserID][itemList[item]] = round(itemAverage[itemList[item]])
            spamTimeProfile[startUserID][itemList[item]]= random.sample(random_time,1)[0] # random time assigned
        for _ in range(targetCount):
            target = np.random.randint(len(targetItems))
            spamProfile[startUserID][targetItems[target]] = targetScore
            spamTimeProfile[startUserID][targetItems[target]]= random.sample(random_time,1)[0] # random time assigned
            spamItem[startUserID].append(targetItems[target])
        startUserID += 1
    print(f"userid={startUserID}")


def getFillerItems():
    mu = int(fillerSize*len(itemProfile))
    sigma = int(0.1*mu)
    markedItemsCount = abs(int(round(random.gauss(mu, sigma))))
    markedItems = np.random.randint(len(itemProfile), size=markedItemsCount)
    return markedItems.tolist()
############################################## average attack ##########################################





In [ ]:
outputDir = "/content/drive/MyDrive/attack_datasets/Movielens100K/average/"
for i in [0.1,0.15,0.2,0.25]:
  attackSize = i
  fillerSize = 0.05
  if i>0.15:
    fillerSize = 0.1
  selectedSize = 0.005
  targetCount = 100
  targetScore = 4.0
  threshold = 3.0
  maxScore = 4.0
  minScore = 1.0
  minCount = 5
  maxCount = 150
  linkSize = 0.001


  itemList = []

  spamProfile = defaultdict(dict)
  spamItem = defaultdict(list)  # items rated by spammers
  spamTimeProfile = defaultdict(dict)
  targetItems = []
  itemAverage = {}
  startUserID = 0
  getAverageRating()
  selectTarget()
  average_attack()
  #attack.farmLink()
  generateLabels(f'labels_{i*10}.txt')
  generateProfiles(f'profiles_{i*10}.txt')

In [ ]:
import pandas as pd
names = ['user_id', 'item_id', 'rating', 'timestamp']
a=pd.read_csv("/content/drive/MyDrive/attack_datasets/Movielens1M/bandwagon_profiles.txt",delim_whitespace=True,names=names)
a

In [ ]:
############################################## bandwagon attack ##########################################
hotItems = sorted(itemProfile.items(), key=lambda d: len(d[1]), reverse=True)[
    : int(selectedSize * len(itemProfile))
]
def bandwagon_attack(startID=0):
    print("Modeling bandwagon attack...")
    startUserID = len(userProfile) if startID == 0 else startID
    for _ in range(int(len(userProfile) * attackSize)):
        fillerItems = getFillerItems()
        for item in fillerItems:
            spamProfile[startUserID][itemList[item]] = random.randint(
                minScore, maxScore
            )
            spamTimeProfile[startUserID][itemList[item]]= random.sample(random_time,1)[0] # random time assigned
        selectedItems = getSelectedItems()
        for item in selectedItems:
            spamProfile[startUserID][item] = targetScore
            spamTimeProfile[startUserID][item]= random.sample(random_time,1)[0] # random time assigned
        for _ in range(targetCount):
            target = np.random.randint(len(targetItems))
            spamProfile[startUserID][targetItems[target]] = targetScore
            spamTimeProfile[startUserID][targetItems[target]]= random.sample(random_time,1)[0] # random time assigned
            spamItem[startUserID].append(targetItems[target])
        startUserID += 1
    print(f"userid={startUserID}")

def getFillerItems():
    mu = int(fillerSize * len(itemProfile))
    sigma = int(0.1 * mu)
    markedItemsCount = int(round(random.gauss(mu, sigma)))
    markedItemsCount = max(markedItemsCount, 0)
    return np.random.randint(len(itemProfile), size=markedItemsCount)


def getSelectedItems():
    mu = int(selectedSize * len(itemProfile))
    sigma = int(0.1 * mu)
    markedItemsCount = abs(int(round(random.gauss(mu, sigma))))
    markedIndexes = np.random.randint(len(hotItems), size=markedItemsCount)
    return [hotItems[index][0] for index in markedIndexes]

In [ ]:
outputDir = "/content/drive/MyDrive/attack_datasets/Movielens100K/bandwagon/"
for i in [0.1,0.15,0.2,0.25]:
  attackSize = i
  fillerSize = 0.05
  if i>0.15:
    fillerSize = 0.1
  selectedSize = 0.005
  targetCount = 100
  targetScore = 4.0
  threshold = 3.0
  maxScore = 4.0
  minScore = 1.0
  minCount = 5
  maxCount = 150
  linkSize = 0.001


  itemList = []

  spamProfile = defaultdict(dict)
  spamItem = defaultdict(list)  # items rated by spammers
  spamTimeProfile = defaultdict(dict)
  targetItems = []
  itemAverage = {}
  startUserID = 0
  getAverageRating()
  selectTarget()
  bandwagon_attack()
  #attack.farmLink()
  generateLabels(f'bandwagon_labels_{i*10}.txt')
  generateProfiles(f'bandwagon_profiles_{i*10}.txt')

In [ ]:
############################################## random attack ##########################################


def random_attack(startID=0):
    print('Modeling random attack...')
    startUserID = len(userProfile) if startID == 0 else startID
    for _ in range(int(len(userProfile)*attackSize)):
        fillerItems = getFillerItems()
        for item in fillerItems:
            spamProfile[startUserID][itemList[item]
                                          ] = random.randint(minScore, maxScore)
            spamTimeProfile[startUserID][itemList[item]]= random.sample(random_time,1)[0] # random time assigned
        for _ in range(targetCount):
            target = np.random.randint(len(targetItems))
            spamProfile[startUserID][targetItems[target]] = targetScore
            spamTimeProfile[startUserID][targetItems[target]]= random.sample(random_time,1)[0] # random time assigned
            spamItem[startUserID].append(targetItems[target])
        startUserID += 1
    print(f"userid={startUserID}")

def getFillerItems():
    mu = int(fillerSize*len(itemProfile))
    sigma = int(0.1*mu)
    markedItemsCount = abs(int(round(random.gauss(mu, sigma))))
    markedItems = np.random.randint(len(itemProfile), size=markedItemsCount)
    return markedItems.tolist()
############################################## random attack ##########################################

In [ ]:
outputDir = "/content/drive/MyDrive/attack_datasets/Movielens100K/random/"
for i in [0.1,0.15,0.2,0.25]:
  attackSize = i
  fillerSize = 0.05
  if i>0.15:
    fillerSize = 0.1
  selectedSize = 0.005
  targetCount = 100
  targetScore = 4.0
  threshold = 3.0
  maxScore = 4.0
  minScore = 1.0
  minCount = 5
  maxCount = 150
  linkSize = 0.001


  itemList = []

  spamProfile = defaultdict(dict)
  spamItem = defaultdict(list)  # items rated by spammers
  spamTimeProfile = defaultdict(dict)
  targetItems = []
  itemAverage = {}
  startUserID = 0
  getAverageRating()
  selectTarget()
  random_attack()
  #attack.farmLink()
  generateLabels(f'random_labels_{i*10}.txt')
  generateProfiles(f'random_profiles_{i*10}.txt')

In [ ]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
ratings=pd.read_csv("/content/drive/MyDrive/attack_datasets/Netflix300K/random/random_profiles_1.0.txt",delim_whitespace=True,names=names)
names1=['user_id','label']
labels=pd.read_csv("/content/drive/MyDrive/attack_datasets/Netflix300K/random/random_labels_1.0.txt",delim_whitespace=True,names=names1)
